<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Знакомство-с-данными" data-toc-modified-id="Знакомство-с-данными-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Знакомство с данными</a></span></li><li><span><a href="#Выборки:-обучающая,-валидационная-и-тестовая" data-toc-modified-id="Выборки:-обучающая,-валидационная-и-тестовая-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Выборки: обучающая, валидационная и тестовая</a></span></li><li><span><a href="#Исследование-моделей" data-toc-modified-id="Исследование-моделей-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Исследование моделей</a></span><ul class="toc-item"><li><span><a href="#Модель-&quot;Решающее-дерево&quot;" data-toc-modified-id="Модель-&quot;Решающее-дерево&quot;-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Модель "Решающее дерево"</a></span></li><li><span><a href="#Модель-&quot;Случайный-лес&quot;" data-toc-modified-id="Модель-&quot;Случайный-лес&quot;-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Модель "Случайный лес"</a></span></li><li><span><a href="#Модель-&quot;Логистическая-регрессия&quot;" data-toc-modified-id="Модель-&quot;Логистическая-регрессия&quot;-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Модель "Логистическая регрессия"</a></span></li></ul></li><li><span><a href="#Проверка-качества-лучшей-модели-на-тестовой-выборке" data-toc-modified-id="Проверка-качества-лучшей-модели-на-тестовой-выборке-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка качества лучшей модели на тестовой выборке</a></span></li><li><span><a href="#Проверка-модели-на-адекватность" data-toc-modified-id="Проверка-модели-на-адекватность-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Проверка модели на адекватность</a></span></li><li><span><a href="#Общий-вывод" data-toc-modified-id="Общий-вывод-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Общий вывод</a></span></li></ul></div>

# Проект: "Рекомендация тарифов"

**Задача проекта:**

Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы. Нужно построить модель для задачи классификации, которая выберет подходящий тариф.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

В рамках проекта мы проведем следующие **этапы исследования**: 

1) Ознакомимся с данными, изучив общую информацию

2) Разобьем данные на выборки: обучающую, валидационную и тестовую

3) Исследуем качество разных моделей (решающего дерева, случайного леса и логистической регрессии), меняя гиперпараметры

4) Проверим качество лучшей модели на тестовой выборке

5) Проверим модели на вменяемость

6) Подведем итоги исследования (общий вывод)

Приступим к исследованию! 

## Знакомство с данными

In [30]:
#импортируем необходимые для проекта библиотеки и алгоритмы:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from random import randint
from sklearn.dummy import DummyClassifier

In [3]:
#прочтем файл: 
df = pd.read_csv('/datasets/users_behavior.csv')
#выведем на экран первые 10 строк таблицы для ознакомления: 
df.head(10)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


Поясним, какие данные содержатся в таблице: 
- сalls — количество звонков,
- minutes — суммарная длительность звонков в минутах,
- messages — количество sms-сообщений,
- mb_used — израсходованный интернет-трафик в Мб,
- is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

In [4]:
#выведем на экран общую информацию о данных таблицы: 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


В таблице нет пропусков, и типы данных соответствуют значениям в таблице. Предобработка нам не потребуется, можем перейти непосредственно к исследованию. 

**Вывод** 

На этапе знакомства с данными мы импортировали необходимые для дальнейшей работы библиотеки и алгоритмы, ознакомились с общей информацией о данных в таблице, выявили, что предобработка данных не потребуется, что существенно экономит нам время, и мы можем перейти к исследованию. 

Теперь нам нужно подготовить выборки из всего массива данных: обучающую, валидационную и тестовую. 

## Выборки: обучающая, валидационная и тестовая

Так как тестовой выборки у нас нет, то нам необходимо **исходные данные разбить на 3 группы**: 

*60% данных- обучающая выборка, 20% - валидационная, 20% - тестовая*. 

С помощью функции `train_test_split` мы сначала отделим 20 % для тестовой выборки,  затем от оставшихся данных отделим еще 25% для валидационной выборки, остальное- обучающая. 

In [5]:
#поделим массив данных на признаки (features_df) и целевой признак (target_df):
features_df = df.drop('is_ultra', axis=1)
target_df = df['is_ultra']
#выделим 20% данных для тестовой выборки(features_test, target_test):
features_without_test, features_test, target_without_test, target_test = train_test_split(
    features_df, target_df, test_size=0.2, random_state=1000000, shuffle=True, stratify=target_df)
#выделим 25% от оставшихся данных для валид-ной выборки(features_valid, target_valid),ост.- обучающая (features_train, target_train):
features_train, features_valid, target_train, target_valid = train_test_split(
    features_without_test, target_without_test, test_size=0.25, random_state=1000000, shuffle=True, stratify=target_without_test)
#для проверки выведем на экран размеры получившихся выборок методом shape: 
display('Размер features и target обучающей выборки:', features_train.shape, target_train.shape)
display('Размер features и target валидационной и тестовой выборок:', features_valid.shape, target_valid.shape, features_test.shape, target_test.shape)


'Размер features и target обучающей выборки:'

(1928, 4)

(1928,)

'Размер features и target валидационной и тестовой выборок:'

(643, 4)

(643,)

(643, 4)

(643,)

**Вывод**

Итак, у нас есть 3 выборки: обучающая (features_train, target_train), тестовая (features_test, target_test) и валидационная (features_valid, target_valid), выделенные из исходного датасета в соотношении 3/1/1. 
И мы готовы к обучению моделей. 

## Исследование моделей

 В рамках нашего проекта мы будем исследовать **3 модели**: 
- *модель решающего дерева*; 
- *модель случайного леса* ;
- *модель логистической регрессии*.

Меняя гиперпараметры, добьемся наибольшего значения `accuracy` и выберем лучшую модель по этому показателю.

Начнем с модели решающего дерева. 

### Модель "Решающее дерево"

In [ ]:
#с помощью модуля GridSearchCV подберем оптимальные гиперпараметры для модели решающего дерева (model1):
model1 = DecisionTreeClassifier(random_state=1000000)
parametrs1 = { 'max_depth': range (1,10),
              'min_samples_leaf': range (1,8),
              'min_samples_split': range (2,10,2) }
grid1 = GridSearchCV(model1, parametrs1, cv=5)
grid1.fit(features_train,target_train)
display(grid1.best_params_)

In [7]:
#перезапишем нашу модель(model1) с оптимальными гиперпараметрами и найдем accuracy(accuracy1):
model1 = DecisionTreeClassifier(random_state=1000000, max_depth=5, min_samples_leaf=1, min_samples_split=2)
model1.fit(features_train,target_train)
accuracy1 = model1.score(features_valid, target_valid).round(4)
display('Accuracy модели решающего дерева на валидационной выборке:', accuracy1)

'Accuracy модели решающего дерева на валидационной выборке:'

0.7854

### Модель "Случайный лес"

In [8]:
#с помощью модуля GridSearchCV подберем оптимальные гиперпараметры для модели случайного леса (model2):
model2 = RandomForestClassifier(random_state=1000000)
parametrs2 = {'n_estimators': range (10, 51, 10),
              'max_depth': range (1,11)}
grid2 = GridSearchCV(model2, parametrs2, cv=5)
grid2.fit(features_train,target_train)
display(grid2.best_params_)

{'max_depth': 9, 'n_estimators': 30}

In [9]:
#перезапишем нашу модель(model2) с оптимальными гиперпараметрами и найдем accuracy(accuracy2):
model2 = RandomForestClassifier(random_state=1000000, n_estimators=30, max_depth=9)
model2.fit(features_train,target_train)
accuracy2 = model2.score(features_valid, target_valid).round(4)
display('Accuracy модели случайного леса на валидационной выборке:', accuracy2)

'Accuracy модели случайного леса на валидационной выборке:'

0.8072

### Модель "Логистическая регрессия"

In [10]:
#обучим модель лог.регрессии ( model3) на обучающей выборке и посчитаем качество(accuracy3) на валидационных данных:
model3 = LogisticRegression(random_state=1000000)
model3.fit(features_train,target_train)
accuracy3 = model3.score(features_valid, target_valid).round(4)
display('Accuracy модели логистической регрессии на валидационной выборке:', accuracy3)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


'Accuracy модели логистической регрессии на валидационной выборке:'

0.7294

**Вывод**

Представим результаты исследования качества моделей в виде таблицы:

In [ ]:
results = pd.DataFrame({'модель' : ['Решающее дерево', 'Случайный лес', 'Логистическая регрессия'],
                       'accuracy' : [accuracy1, accuracy2, accuracy3]})
results.sort_values(by='accuracy', ascending=False)

Таким образом, модели "Случайный лес" и "Решающее дерево" по значению `accuracy` на валидационной выборке показали результат гораздо лучше, чем модель "Логистическая регрессия". По условию задачи доля правильных ответов не может быть ниже 0,75, поэтому для проверки качества на тестовой выборке мы возьмем модели "Случайный лес" и "Решающее дерево"(model1 и model2).

## Проверка качества лучшей модели на тестовой выборке

Проверим модели "Случайный лес" и "Решающее дерево" на тестовой выборке, найдем значения accuracy. 

In [12]:
#найдем значение accuracy на тестовой выборке для моделей "Случайный лес" и "Решающее дерево"(accuracy2_test, accuracy1_test):
accuracy2_test = model2.score(features_test, target_test).round(4)
display('Accuracy модели случайного леса на тестовой выборке:', accuracy2_test)
accuracy1_test = model1.score(features_test, target_test).round(4)
display('Accuracy модели решающего дерева на тестовой выборке:', accuracy1_test)

'Accuracy модели случайного леса на тестовой выборке:'

0.8087

'Accuracy модели решающего дерева на тестовой выборке:'

0.7947

**Вывод**

Модель "Случайный лес" с гиперпараметрами 'max_depth'= 9 и 'n_estimators'= 30 показала наилучший результат. Модель дает верные предсказания в более чем 80% случаев. Это хороший результат. 

Теперь проверим нашу модель "на адекватность". Убедимся, что наша модель работает лучше, чем "подбрасывание монеты".

Проведем обучение модели на объединенных данных обучающей и валидационной выборок, а затем посчитаем показатель Accuracy на тестовой выборке.

In [35]:
#обучим модель слчайного леса (model2) на объединенных данных обучающей и валидационной выборок (features_without_test, target_without_test):
model2.fit(features_without_test, target_without_test)
#найдем значение accuracy на тестовой выборке для модели "Случайный лес":
accuracy2_test = model2.score(features_test, target_test).round(4)
display('Accuracy модели случайного леса на тестовой выборке:', accuracy2_test)

'Accuracy модели случайного леса на тестовой выборке:'

0.8009

После переобучения модели на объединенных данных качество осталось практически на том же уровне.

## Проверка модели на адекватность

Итак, нам необходимо сравнить результат работы нашей модели с тем, если бы мы проставляли 0 и 1 наугад,например, подбрасывая монету. 

In [13]:
#создадим список с рандомными предсказаниями 0 и 1 (random_predictions):
random_predictions = []
for _ in range(643):
    random_predictions.append(randint(0,1))
#выведем на экран значение accuracy_score по данным target_test и random_predictions, а также target_valid и random_predictions : 
display('Accuracy_score по данным target_test и random_predictions', accuracy_score(target_test, random_predictions).round(4))
display('Accuracy_score по данным target_valid и random_predictions', accuracy_score(target_valid, random_predictions).round(4))


'Accuracy_score по данным target_test и random_predictions'

0.4946

'Accuracy_score по данным target_valid и random_predictions'

0.535

**Вывод**

Представим результаты полученного значения accuracy в виде таблицы: 

In [14]:
results1 = pd.DataFrame({'выборки' : ['тестовая', 'валидационная'],
                         'accuracy модели случайного леса' : [accuracy2_test, accuracy2],
                         'accuracy c подбрасыванием монеты' : [accuracy_score(target_test, random_predictions).round(4), accuracy_score(target_valid, random_predictions).round(4)]})
results1

,выборки,accuracy модели случайного леса,accuracy c подбрасыванием монеты
0,тестовая,0.8087,0.4946
1,валидационная,0.8072,0.5350


Результаты в таблице очень наглядно показывают нам, что в случае проставления тарифа путем "подбрасывания монеты" вероятность успеха чуть больше 50%, то есть 50/50 угадаем мы таким образом тариф или нет. Обученная же модель предсказывает верный тариф с вероятностью более 80 %.

Теперь проверимэффективность предсказаний модели, которая предсказывает всегда только 1 класс ( или 1, или 0), наиболее часто встречающийся в выборке. 

Проверим, каких значений больше в столбце `is_ultra`:

In [33]:
#с помощью метода value_counts() посмотрим на распределение 0 и 1 в столбце is_ultra нашего датасета: 
df['is_ultra'].value_counts()

0    2229
1     985
Name: is_ultra, dtype: int64

Так как 0 больше, с помощью инструмента DummyClassifier создадим модель, котора будет предсказывать только 0 и посчитаем значение Accuracy_score:

In [31]:
dummy_model = DummyClassifier(strategy="constant", random_state=1000000, constant=0)
dummy_model.fit(features_train, target_train)
dummy_predictions0 = dummy_model.predict(features_valid)
#выведем на экран значение accuracy_score по данным target_test и random_predictions, а также target_valid и random_predictions : 
display('Accuracy_score по данным target_test и dummy_predictions0', accuracy_score(target_test, dummy_predictions).round(4))
display('Accuracy_score по данным target_valid и dummy_predictions0', accuracy_score(target_valid, dummy_predictions).round(4))


'Accuracy_score по данным target_test и dummy_predictions0'

0.6936

'Accuracy_score по данным target_valid и dummy_predictions0'

0.6936

Наша фиктивная модель, предсказывающая всегда только 0, показала уровень качества 69% и не может быть использована в качестве рекомендательной модели, так как качество низкое, и оно обучловлено только текущим распределением предпочтений пользоватлей по тарифам в представленной выборке, при изменении предпочтений пользователей эта модель перестанет работать.  

Обученная же модель выдает показатели accuracy более 80 %, ее мы можем рекомендовать для дальнейшего использования. 

## Общий вывод

Для подведения итогов исследования давайте еще раз вспомним задачу, которая стояла перед нами в начале исследования: 

Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».

В рамках исследовнаия мы взяли данные о поведении клиентов, которые уже перешли на эти тарифы, поделили их на 3 выборки: обучающую, валидационную и тестовую, и обучили по данным обучающей выборки 3 модели, подобрав им оптимальные гиперпараметры. 

Было выявлено, что модели "Случайный лес" и "Решающее дерево" по значению accuracy на валидационной выборке показали результат гораздо лучше, чем модель "Логистическая регрессия". По условию задачи доля правильных ответов не может быть ниже 0,75, поэтому для проверки качества на тестовой выборке мы взяли модели "Случайный лес" и "Решающее дерево". 

При проверке на тестовой выборке были получены почти такие же результаты, поэтому сделан вывод, что модель "Случайный лес" с гиперпараметрами 'max_depth'= 9 и 'n_estimators'= 30 показала наилучший результат. 
Модель дала верные предсказания в более чем 80% случаев. И это хороший результат.

С целью проверки модели на адекватность мы сравнили результаты качества нашей модели с тем, если бы мы проставляли 0 и 1 наугад,например, подбрасывая монету. И выявили, что в случае проставления тарифа путем "подбрасывания монеты" вероятность успеха чуть больше 50%, то есть 50/50 угадаем мы таким образом тариф или нет. Обученная же модель предсказывает верный тариф с вероятностью более 80 %.

Подводя итог всего вышесказанного, оператору мобильной связи "Мегалайн" мы бы рекомендовали с целью построения системы автоматического предложения клиентам новых тарифов «Смарт» или «Ультра» использовать модель "Случайный лес" из нашего исследования.